## optimisation algos

In [1]:
# Importer les bibliothèques nécessaires

import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, confusion_matrix,accuracy_score
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import missingno as msno
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns
import imblearn as imb
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from sklearn.metrics import auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from imblearn import FunctionSampler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import mlflow
import mlflow.sklearn


In [2]:
mlflow.set_tracking_uri("http://localhost:5000")

In [3]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base=pd.read_csv("/content/drive/MyDrive/Notebooks/P7/basep7.csv")
        
except ModuleNotFoundError :
    
    base = pd.read_csv("basep7.csv")
    
base = base.drop( columns = ['Unnamed: 0'])

In [4]:
# one_hot_encoder classique pour les non numériques
def one_hot_encoder(base, nan_as_category = True):
    original_columns = list(base.columns)
    categorical_columns = [col for col in base.columns if base[col].dtype == 'object']
    base2 = pd.get_dummies(base, columns= categorical_columns, dummy_na= True)
    new_columns = [c for c in base.columns if c not in original_columns]
    return base2
print (base.shape)
base2 =one_hot_encoder(base)
print (base2.shape)
base = base2

(307511, 122)
(307511, 262)


In [5]:
# Remplacer les valeurs manquantes par la moyenne de la colonne
base = base.fillna(base.mean())
base= base.sample(500)

In [6]:
# Séparer les variables explicatives (X) et la variable cible (y)
X = base.drop("TARGET", axis=1)
y = base["TARGET"]



In [7]:
# scaler 
col_names=X.select_dtypes(include='number').columns.tolist()
features = X[col_names]
scaler = StandardScaler().fit(features.values)
features_scale = scaler.transform(features.values)
X[col_names] = features_scale
print (X.shape) 
del features
del features_scale

(500, 261)


In [8]:
def custom_score_func(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    score = tp + tn - fp -( 10 * fn)
    return score


# Création du scorer personnalisé
score_metier= make_scorer(custom_score_func)

scorer = score_metier

In [9]:
debut = time.time()

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 



In [11]:
#X_resampled, y_resampled = SMOTETomek(random_state=22).fit_resample(X_train,y_train)

In [64]:
def comp_algos(X,y,algos,params,cvi,scorer,sampling) : 

    
    roc_aucs = []
    confusions = []
    Report_df = pd.DataFrame(columns=['Algorithme', 'score nom ','score']) 
    Append = []

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
    
    
    
    
    
    for i in range(len(algos)): 
        with mlflow.start_run(experiment_id=experiment_id):          
                   
            #faire boucle sur le sampling 
            for s in range(len(sampling)): 
                with mlflow.start_run(run_name="Child Run", nested=True):
                #with mlflow.start_run(run_name="Child Run", nested=True):
            
                    # Apply sampling            
                    debut = time.time()
                    X_resampled, y_resampled = sampling[s].fit_resample(X_train,y_train)

# Créer un objet GridSearchCV qui va comparer les performances des algorithmes sur les données en utilisant le score roc_auc comme critère d'évaluation 
                    print (algos[i], params[i] )          
                    grid = GridSearchCV(estimator=algos[i], param_grid=params[i], scoring=scorer, cv=cvi, refit= True )
                    grid.fit(X_resampled,y_resampled)
                    del X_resampled
                    del y_resampled
            
                    """# Afficher le meilleur score et le meilleur algorithme trouvé par GridSearchCV 
                    print("Meilleur score:", grid.best_score_)
                    print("Meilleur algorithme:", grid.best_estimator_)
                    #print ( "cv results ;", grid.cv_results_ ) 
                    print ( "best_params_",grid.best_params_)
                    print ("best_index_",grid.best_index_)
                    print ("scorer_",grid.scorer_)
                    print ("n_splits_",grid.n_splits_)
                    print ("sampling method",sampling[s])"""
                        
                    cv_result= pd.DataFrame(grid.cv_results_)
                    display ( cv_result )

# Prédire les classes pour chaque algorithme 
                    
                    pred_y_train = grid.predict(X_train)
                    score = accuracy_score(y_train,pred_y_train)
                    print ('accuracy train',score) 
          
                    roc_auc_train = roc_auc_score(y_train, pred_y_train)
                    #print ( 'roc_auc_train',roc_auc_train)
                    print(f'roc_auc_train: {roc_auc_train:.1%}')

                    pred_y_train_proba=grid.predict_proba(X_train)[:,1]
                    roc_auc_discret_train = roc_auc_score(y_train, pred_y_train_proba)
                    print( "Roc auc train predict proba",roc_auc_discret_train)
        
                    pred_y_test = grid.predict(X_test)
                    score = accuracy_score(y_test,pred_y_test)
                    print ('accuracy test',score)
          
                    roc_auc_test = roc_auc_score(y_test, pred_y_test)
                    print ( 'roc_auc_ test ',roc_auc_test)

                    pred_y_test_proba=grid.predict_proba(X_test)[:,1]
                    roc_auc_discret_test= roc_auc_score(y_test, pred_y_test_proba)
                    print( "Roc auc score test predict proba",roc_auc_discret_test)

# Calculer la matrice de confusion pour chaque algorithme

                    matrice_confusion = confusion_matrix(y_true=y_train,y_pred=pred_y_train)
                    tn, fp, fn, tp = confusion_matrix(y_true=y_train,y_pred=pred_y_train, normalize='all').ravel()

           
# Créer un dataframe pandas qui contient les résultats de chaque algorithme
                    print(' score ', scorer,':' ,grid.best_score_)
                    duration = (time.time() - debut )/60
                    print (duration ,'minutes')
                    Append = pd.DataFrame([[ algos[i],grid.best_params_,scorer ,grid.best_score_,sampling[s],roc_auc_train,roc_auc_discret_train
                                    ,roc_auc_test,roc_auc_discret_test,tn,fp,fn,tp,duration]]
                                  ,columns=['Algorithme','Meilleur param', 'score nom ','best score','sampling','roc_auc_train'
                                            ,'Roc_auc_train_discret','roc_auc_test','roc_auc_discret_test','TN','FP','FN','TP','minutes'])
                    Report_df = pd.concat([Report_df, Append]) 
            
                    #mlflow.log_param("n_estimators", grid_search.best_params_['random_forest_classifier__n_estimators'])
                    #mlflow.log_param("max_depth", grid_search.best_params_['random_forest_classifier__max_depth'])
                    #mlflow.log_param("min_samples_leaf", grid_search.best_params_['random_forest_classifier__min_samples_leaf'])
                    #mlflow.log_param("best_algos_", grid.best_estimator_)
                    #mlflow.log_param("best_params_", grid.best_params_)
                    mlflow.log_param("scorer", grid.scorer_)
                    mlflow.log_param("scorer name", scorer)
                    mlflow.log_param("sampling", sampling[s])
                    mlflow.log_metric("best Score", grid.best_score_)
                    mlflow.log_metric("roc auc proba train", roc_auc_discret_train)
                    
                mlflow.sklearn.log_model(algos[i], "algo_model")
                mlflow.log_param("algo_param", algos[i])
                mlflow.sklearn.log_model(grid.best_estimator_, "model best estimator")
                mlflow.end_run()
        
    return Report_df

In [67]:
algos = [ LogisticRegression(random_state=22),DecisionTreeClassifier(random_state=22), 
         XGBClassifier(random_state=22),RandomForestClassifier(random_state=22), GaussianNB(),DummyClassifier()]


# Définir une grille de paramètres à optimiser pour chaque algorithme
params = [
    
    {"C": [0.01, 0.1, 1, 10], "penalty": ["l2", None]}, # pour LogisticRegression ‘l1’, ‘l2’, ‘elasticnet’, None
    {"max_depth": [3, 5, 7], "min_samples_leaf": [5, 10, 15]}, # pour DecisionTreeClassifier
    {"n_estimators": [100, 200], "max_depth": [3, 5]}, # pour RandomForestClassifier 
    {}, # rien pour XGBClassifier car trop de parametre taille > 500 c : plante 
    {} ,# pas de paramètres à optimiser pour GaussianNB 
    {"strategy": ["most_frequent","prior","stratified","uniform"]}, # pour DummyClassifier
    ]




cvi =3

"""algos = [ DecisionTreeClassifier(random_state=22)]
params = [ {"max_depth": [3], "min_samples_leaf": [5]}] # pour DecisionTreeClassifier"""

def custom_score_func(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    score = tp + tn - fp -( 10 * fn)
    #score 2 = tp + (10 * tn )- fp -(10 * fn)
    return score


# Création du scorer personnalisé
score_metier= make_scorer(custom_score_func)

scorer = score_metier

def nosamplerfunc(X, y):
    return X[:], y[:]
nosampler = FunctionSampler(func=nosamplerfunc)

sampling = [nosampler, RandomOverSampler(random_state=22),RandomUnderSampler(random_state=22),SMOTETomek(random_state=22)]

expe = "test_230420_21"
experiment_id = mlflow.create_experiment(expe)

result_metier= comp_algos(X,y,algos,params,cvi,scorer,sampling)
result_metier

result_metier.to_excel("resultat_metier_flow_.xls")

LogisticRegression(random_state=22) {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2', None]}


C:\Users\Hugues\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Hugues\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Hugues\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\Hugues\AppData\Roaming\Python\Python310\site-packages\sklearn\l

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.015633,0.000018,0.000000,0.000000,0.01,l2,"{'C': 0.01, 'penalty': 'l2'}",2.0,1.0,1.0,1.333333,0.471405,2
1,0.005195,0.007346,0.000000,0.000000,0.01,None,"{'C': 0.01, 'penalty': None}",NaN,NaN,NaN,NaN,NaN,5
2,0.015624,0.000003,0.005207,0.007364,0.1,l2,"{'C': 0.1, 'penalty': 'l2'}",-6.0,12.0,6.0,4.000000,7.483315,1
3,0.000000,0.000000,0.000000,0.000000,0.1,None,"{'C': 0.1, 'penalty': None}",NaN,NaN,NaN,NaN,NaN,6
4,0.028206,0.004298,0.005207,0.007364,1,l2,"{'C': 1, 'penalty': 'l2'}",-16.0,-8.0,-10.0,-11.333333,3.399346,3
5,0.000000,0.000000,0.000000,0.000000,1,None,"{'C': 1, 'penalty': None}",NaN,NaN,NaN,NaN,NaN,7
6,0.040790,0.004295,0.000000,0.000000,10,l2,"{'C': 10, 'penalty': 'l2'}",-22.0,-10.0,-16.0,-16.000000,4.898979,4
7,0.000000,0.000000,0.000000,0.000000,10,None,"{'C': 10, 'penalty': None}",NaN,NaN,NaN,NaN,NaN,8


accuracy train 0.955
roc_auc_train: 75.0%
Roc auc train predict proba 0.9926739926739927
accuracy test 0.95
roc_auc_ test  0.5833333333333334
Roc auc score test predict proba 0.43794326241134746
 score  make_scorer(custom_score_func) : 4.0
0.007860565185546875 minutes
LogisticRegression(random_state=22) {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2', None]}


C:\Users\Hugues\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Hugues\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

...

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.021327,0.005494,0.006178,0.001060,3,5,"{'max_depth': 3, 'min_samples_leaf': 5}",-222.0,73.0,35.0,-38.000000,131.029259,5
1,0.012849,0.002158,0.007102,0.000711,3,10,"{'max_depth': 3, 'min_samples_leaf': 10}",-286.0,73.0,5.0,-69.333333,155.701281,9
2,0.015918,0.004761,0.005280,0.002001,3,15,"{'max_depth': 3, 'min_samples_leaf': 15}",-268.0,73.0,13.0,-60.666667,148.639011,8
3,0.021873,0.001791,0.005170,0.001673,5,5,"{'max_depth': 5, 'min_samples_leaf': 5}",-122.0,197.0,114.0,63.000000,135.132034,2
4,0.040074,0.003891,0.011391,0.005543,5,10,"{'max_depth': 5, 'min_samples_leaf': 10}",-241.0,39.0,32.0,-56.666667,130.374674,7
5,0.043543,0.032959,0.006273,0.001147,5,15,"{'max_depth': 5, 'min_samples_leaf': 15}",-210.0,112.0,47.0,-17.000000,139.027575,3
6,0.023259,0.001215,0.004002,0.000832,7,5,"{'max_depth': 7, 'min_samples_leaf': 5}",-119.0,186.0,124.0,63.666667,131.621511,1
7,0.024966,0.012654,0.004655,0.001245,7,10,"{'max_depth': 7, 'min_samples_leaf': 10}",-212.0,39.0,32.0,-47.000000,116.707612,6
8,0.020486,0.004827,0.005185,0.001693,7,15,"{'max_depth': 7, 'min_samples_leaf': 15}",-210.0,112.0,47.0,-17.000000,139.027575,3


accuracy train 0.9275
roc_auc_train: 76.0%
Roc auc train predict proba 0.9239163614163615
accuracy test 0.77
roc_auc_ test  0.48758865248226957
Roc auc score test predict proba 0.4157801418439716
 score  make_scorer(custom_score_func) : 63.666666666666664
0.0172067125638326 minutes
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=22,
              reg_alpha=None, 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.578718,0.260303,0.014976,0.000516,3,100,"{'max_depth': 3, 'n_estimators': 100}",-4.0,1.0,-1.0,-1.333333,2.054805,1
1,0.669262,0.090135,0.013299,0.001881,3,200,"{'max_depth': 3, 'n_estimators': 200}",-6.0,-1.0,-1.0,-2.666667,2.357023,4
2,0.332245,0.008320,0.015025,0.000634,5,100,"{'max_depth': 5, 'n_estimators': 100}",-6.0,1.0,1.0,-1.333333,3.299832,1
3,0.595876,0.028825,0.014302,0.000314,5,200,"{'max_depth': 5, 'n_estimators': 200}",-6.0,1.0,1.0,-1.333333,3.299832,1


accuracy train 1.0
roc_auc_train: 100.0%
Roc auc train predict proba 1.0
accuracy test 0.94
roc_auc_ test  0.5
Roc auc score test predict proba 0.7393617021276596
 score  make_scorer(custom_score_func) : -1.3333333333333333
0.12308202981948853 minutes


MlflowException: INVALID_PARAMETER_VALUE: Param value 'XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma' had length 738, which exceeded length limit of 500

The cause of this error is typically due to repeated calls
to an individual run_id event logging.

Incorrect Example:
---------------------------------------
with mlflow.start_run():
    mlflow.log_param("depth", 3)
    mlflow.log_param("depth", 5)
---------------------------------------

Which will throw an MlflowException for overwriting a
logged parameter.

Correct Example:
---------------------------------------
with mlflow.start_run():
    with mlflow.start_run(nested=True):
        mlflow.log_param("depth", 3)
    with mlflow.start_run(nested=True):
        mlflow.log_param("depth", 5)
---------------------------------------

W

In [ ]:
STOP

In [ ]:


# set tracking URI  - where MLflows saves runs
# the format : "file://" + absolute path

mlflow.set_tracking_uri("http://localhost:5000")
#mlflow.set_tracking_uri("./mlruns")
#lflow.set_tracking_uri("./test")

In [ ]:
mlflow.end_run()

In [ ]:
# create an experiment
experiment_id = mlflow.create_experiment("experiment_9")

In [ ]:
# Création du pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('random_forest_classifier', RandomForestClassifier())
])

clf = RandomForestClassifier()

# Définition de la grille des hyperparamètres à tester
param_grid = {
    'random_forest_classifier__n_estimators': [10, 50, 100],
    'random_forest_classifier__max_depth': [5, 10, 20],
    'random_forest_classifier__min_samples_leaf': [1, 5, 10]
}
# Création de l'objet GridSearchCV
grid_search = GridSearchCV(
    pipeline,param_grid=param_grid, cv=3,scoring=scorer, n_jobs=-1, refit=True)

# Début du suivi de la performance avec MLflow
with mlflow.start_run(experiment_id=experiment_id):

    # Entraînement du modèle avec GridSearchCV
    grid_search.fit(X_resampled, y_resampled)

    # Prédiction sur l'ensemble de test
    y_pred = grid_search.predict(X_test)

  

    # Enregistrement des métriques et des paramètres avec MLflow
    
    mlflow.log_param("n_estimators", grid_search.best_params_['random_forest_classifier__n_estimators'])
    mlflow.log_param("max_depth", grid_search.best_params_['random_forest_classifier__max_depth'])
    mlflow.log_param("min_samples_leaf", grid_search.best_params_['random_forest_classifier__min_samples_leaf'])
    mlflow.log_metric("F1 score", grid_search.best_score_)
  

In [ ]:
 # Enregistrement du modèle avec MLflow
#    mlflow.sklearn.log_model(
#        grid_search.best_estimator_, "models")
# Enregistrement du modèle avec MLflow
mlflow.sklearn.log_model(clf, "RandomForestClassifier")

In [ ]:
# Enregistrement du modèle avec MLflow
mlflow.sklearn.log_model(
        grid_search.best_estimator_, "models")

In [ ]:
#model= RandomForestClassifier(n_estimators=100, max_depth=5, random_state=22, scorer=accuracy_score)

rfc= RandomForestClassifier(n_estimators=100, max_depth=5, random_state=22)
model = GridSearchCV(estimator=rfc, param_grid=[{"n_estimators": [100], "max_depth": [5]}], scoring=scorer, cv=3, refit= True )

model.fit(X_resampled,y_resampled)

In [ ]:
print ( time.time()-debut)